In [8]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import tensorflow.keras as keras
import tensorflow  as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import *
from tensorflow.keras.optimizers import *


from sklearn.metrics import mean_absolute_error
from sklearn.compose import make_column_transformer
from sklearn.preprocessing import MinMaxScaler, OneHotEncoder
from sklearn.model_selection import train_test_split

from xgboost import XGBRegressor
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor

from tensorflow.keras.callbacks import *
import os

In [151]:
#Daten von einem Monat (08/2018) mit Wetter und echten Ausleih und Rückgabe-Werten
frame_alr = pd.read_csv(r'20152018/wetter_mit_ausleihen_rueckgaben_mit_dummy_082018.csv')

df_alr = pd.DataFrame(frame_alr)

df_alr2 = pd.DataFrame(df_alr)
df_alr.head()

,tempC,precipMM,humidity,windspeedKmph,year,day,holidays,workday,rueckgaben,ausleihen,...,month_3,month_4,month_5,month_6,month_7,month_8,month_9,month_10,month_11,month_12
0,24,0.0,83,11,2018,1,0,1,2,2,...,0,0,0,0,0,1,0,0,0,0
1,24,0.0,84,10,2018,1,0,1,0,0,...,0,0,0,0,0,1,0,0,0,0
2,24,0.3,86,9,2018,1,0,1,2,1,...,0,0,0,0,0,1,0,0,0,0
3,24,0.4,87,8,2018,1,0,1,0,0,...,0,0,0,0,0,1,0,0,0,0
4,24,0.2,86,9,2018,1,0,1,0,2,...,0,0,0,0,0,1,0,0,0,0


In [152]:
#Daten 2015-2018 Ausleihen mit Wetter
frame_ausleihen = pd.read_csv(r'20152018/ausleihen_merged_ohne_nullen.csv')

df_ausleihen = pd.DataFrame(frame_ausleihen)

df_ausleihen = pd.get_dummies(df_ausleihen, columns=['weekday', 'hour','weatherDesc2','month'] )
df_ausleihen = df_ausleihen.drop('day',axis=1)
#df_ausleihen.head()

In [153]:
df_alr = pd.DataFrame(df_alr,columns=['year', 'holidays', 'workday', 'tempC', 'precipMM', 'humidity',
       'windspeedKmph', 'weekday_0', 'weekday_1', 'weekday_2', 'weekday_3',
       'weekday_4', 'weekday_5', 'weekday_6', 'hour_0', 'hour_1', 'hour_2',
       'hour_3', 'hour_4', 'hour_5', 'hour_6', 'hour_7', 'hour_8', 'hour_9',
       'hour_10', 'hour_11', 'hour_12', 'hour_13', 'hour_14', 'hour_15',
       'hour_16', 'hour_17', 'hour_18', 'hour_19', 'hour_20', 'hour_21',
       'hour_22', 'hour_23', 'weatherDesc2_Blizzard',
       'weatherDesc2_Blowing snow', 'weatherDesc2_Clear',
       'weatherDesc2_Cloudy', 'weatherDesc2_Fog',
       'weatherDesc2_Freezing drizzle', 'weatherDesc2_Heavy freezing drizzle',
       'weatherDesc2_Heavy rain', 'weatherDesc2_Heavy rain at times',
       'weatherDesc2_Heavy snow', 'weatherDesc2_Ice pellets',
       'weatherDesc2_Light drizzle', 'weatherDesc2_Light freezing rain',
       'weatherDesc2_Light rain', 'weatherDesc2_Light rain shower',
       'weatherDesc2_Light sleet', 'weatherDesc2_Light sleet showers',
       'weatherDesc2_Light snow', 'weatherDesc2_Light snow showers',
       'weatherDesc2_Mist', 'weatherDesc2_Moderate or heavy freezing rain',
       'weatherDesc2_Moderate or heavy rain shower',
       'weatherDesc2_Moderate or heavy rain with thunder',
       'weatherDesc2_Moderate or heavy sleet',
       'weatherDesc2_Moderate or heavy snow showers',
       'weatherDesc2_Moderate rain', 'weatherDesc2_Moderate rain at times',
       'weatherDesc2_Moderate snow', 'weatherDesc2_Overcast',
       'weatherDesc2_Partly cloudy', 'weatherDesc2_Patchy heavy snow',
       'weatherDesc2_Patchy light drizzle', 'weatherDesc2_Patchy light rain',
       'weatherDesc2_Patchy light rain with thunder',
       'weatherDesc2_Patchy light snow', 'weatherDesc2_Patchy moderate snow',
       'weatherDesc2_Patchy rain possible',
       'weatherDesc2_Patchy sleet possible',
       'weatherDesc2_Patchy snow possible', 'weatherDesc2_Sunny',
       'weatherDesc2_Thundery outbreaks possible',
       'weatherDesc2_Torrential rain shower', 'month_1', 'month_2', 'month_3',
       'month_4', 'month_5', 'month_6', 'month_7', 'month_8', 'month_9',
       'month_10', 'month_11', 'month_12'])

In [154]:
df_alr.head()

,year,holidays,workday,tempC,precipMM,humidity,windspeedKmph,weekday_0,weekday_1,weekday_2,...,month_3,month_4,month_5,month_6,month_7,month_8,month_9,month_10,month_11,month_12
0,2018,0,1,24,0.0,83,11,0,0,1,...,0,0,0,0,0,1,0,0,0,0
1,2018,0,1,24,0.0,84,10,0,0,1,...,0,0,0,0,0,1,0,0,0,0
2,2018,0,1,24,0.3,86,9,0,0,1,...,0,0,0,0,0,1,0,0,0,0
3,2018,0,1,24,0.4,87,8,0,0,1,...,0,0,0,0,0,1,0,0,0,0
4,2018,0,1,24,0.2,86,9,0,0,1,...,0,0,0,0,0,1,0,0,0,0


In [18]:
#Daten 2015-2018 Rückgaben mit Wetter
frame_rueckgaben = pd.read_csv(r'20152018/rueckgaben_merged_ohne_nullen.csv')

df_rueckgaben = pd.DataFrame(frame_rueckgaben)
#df_rueckgaben.head()

In [35]:
#df_alr = pd.get_dummies(df_alr, columns=['weekday', 'hour','weatherDesc2','month'] )

In [82]:
df_alr = df_alr.drop(['day','rueckgaben','ausleihen'],axis=1)

In [ ]:
df_alr = df_alr.drop('weatherDesc2_Patchy freezing drizzle possible',axis = 1)

In [155]:
#Split Data
#y = df_alr['ausleihen']
#X = df_alr.drop('ausleihen',axis=1)

y = df_ausleihen['count']
X = df_ausleihen.drop('count',axis=1)

# Train-Test-Split
X_train , X_test, y_train, y_test = train_test_split(X,y,train_size=0.8,shuffle=False  )

In [120]:
scaler = MinMaxScaler()

In [121]:
scaler.fit(X_train[["year","tempC","precipMM",'windspeedKmph',"humidity"]])

MinMaxScaler()

In [122]:
df_alr_transformed = pd.DataFrame(df_alr)

In [123]:
df_alr_transformed[["year","tempC","precipMM",'windspeedKmph',"humidity"]] = scaler.transform(df_alr[["year","tempC","precipMM",'windspeedKmph',"humidity"]])

In [94]:
df_alr_transformed

array([[1.        , 0.80392157, 0.        , 0.21568627, 0.79518072],
       [1.        , 0.80392157, 0.        , 0.19607843, 0.80722892],
       [1.        , 0.80392157, 0.02173913, 0.17647059, 0.8313253 ],
       ...,
       [1.        , 0.76470588, 0.        , 0.37254902, 0.63855422],
       [1.        , 0.76470588, 0.        , 0.35294118, 0.65060241],
       [1.        , 0.76470588, 0.        , 0.35294118, 0.6746988 ]])

In [156]:
ct = make_column_transformer(
 (MinMaxScaler(),["year","tempC","precipMM",'windspeedKmph',"humidity"]) , remainder="passthrough" #,
 )

#X_train[['year','tempC','precipMM','humidity']] = MinMaxScaler().fit_transform(X_train[['year','tempC','precipMM','humidity']])


ct.fit(X_train)

df_alr_normalized = ct.transform(df_alr)

#x_train_normalized = ct.transform(X_train)
#x_test_normalized = ct.transform(X_test)

In [149]:
x_test_normalized

array([[1.        , 0.41176471, 0.        , ..., 0.        , 0.        ,
        0.        ],
       [1.        , 0.41176471, 0.        , ..., 0.        , 0.        ,
        0.        ],
       [1.        , 0.41176471, 0.00724638, ..., 0.        , 0.        ,
        0.        ],
       ...,
       [1.        , 0.45098039, 0.10144928, ..., 0.        , 0.        ,
        1.        ],
       [1.        , 0.45098039, 0.05072464, ..., 0.        , 0.        ,
        1.        ],
       [1.        , 0.47058824, 0.36231884, ..., 0.        , 0.        ,
        1.        ]])

In [157]:
model = keras.models.load_model('20152018/ausleihen_modell1')

In [158]:
predicted_ausleihen = model.predict(df_alr_normalized)

In [159]:
df_alr2['ausleihenpredict'] = predicted_ausleihen

In [162]:
df_alr2.to_csv('20152018/ausleihen_predicted.csv',index=False)

In [163]:
df_alr2

,tempC,precipMM,humidity,windspeedKmph,year,day,holidays,workday,rueckgaben,ausleihen,...,month_4,month_5,month_6,month_7,month_8,month_9,month_10,month_11,month_12,ausleihenpredict
0,24,0.0,83,11,2018,1,0,1,2,2,...,0,0,0,0,1,0,0,0,0,1.208710
1,24,0.0,84,10,2018,1,0,1,0,0,...,0,0,0,0,1,0,0,0,0,1.287797
2,24,0.3,86,9,2018,1,0,1,2,1,...,0,0,0,0,1,0,0,0,0,1.060808
3,24,0.4,87,8,2018,1,0,1,0,0,...,0,0,0,0,1,0,0,0,0,0.921061
4,24,0.2,86,9,2018,1,0,1,0,2,...,0,0,0,0,1,0,0,0,0,1.016769
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
739,23,0.2,68,19,2018,31,0,1,14,15,...,0,0,0,0,1,0,0,0,0,25.805307
740,22,0.0,69,19,2018,31,0,1,9,11,...,0,0,0,0,1,0,0,0,0,16.487341
741,22,0.0,70,19,2018,31,0,1,8,7,...,0,0,0,0,1,0,0,0,0,9.240326
742,22,0.0,71,18,2018,31,0,1,7,8,...,0,0,0,0,1,0,0,0,0,5.256702


In [128]:
df_rueck = pd.read_csv(r'20152018/rueckgaben_mit_ausleihen_merged.csv')

In [135]:
df_rueck.head(10)

,year,month,day,hour,count,holidays,weekday,workday,tempC,precipMM,humidity,windspeedKmph,weatherDesc2,ausleihen
24232,2018,8,1,0,2,0,2,1,24,0.0,83,11,Overcast,2.222233
24233,2018,8,1,2,2,0,2,1,24,0.3,86,9,Light rain shower,0.906377
24234,2018,8,1,5,3,0,2,1,25,0.8,85,10,Light rain shower,1.049075
24235,2018,8,1,6,4,0,2,1,25,1.1,83,12,Light rain shower,3.686494
24236,2018,8,1,7,12,0,2,1,26,0.6,79,14,Light rain shower,17.914927
24237,2018,8,1,8,44,0,2,1,27,0.8,74,16,Light rain shower,33.040900
24238,2018,8,1,9,29,0,2,1,28,1.1,70,18,Light rain shower,23.247295
24239,2018,8,1,10,13,0,2,1,28,0.5,69,18,Light rain shower,11.091673
24240,2018,8,1,11,12,0,2,1,28,1.6,69,18,Moderate or heavy rain shower,9.366283
24241,2018,8,1,12,12,0,2,1,29,2.2,68,18,Moderate or heavy rain shower,11.161324


In [131]:
df_rueck = df_rueck[(df_rueck['year']== 2018) & (df_rueck['month'] == 8)]

In [143]:
df_rueck['ausleihen'].mean()

20.991987113750003